Ejercicio 1 - Machine Learning
Mercado Pago en días recientes implementó una campaña de marketing
relacionada con un nuevo producto de Mercado Crédito. Esta campaña está basada
en el marketing directo (llamadas telefónicas) y pretende entender el
comportamiento de los clientes entrevistados a fin de construir modelos que
permitan predecir el comportamiento de clientes no conocidos. Para ello MELI ha
recolectado un conjunto de datos que pretende ser utilizado para la construcción de
modelos de Machine Learning. Una vez recolectados y analizados estos datos se
requiere implementar una Proof of Concept para un modelo que utilice la
información suministrada para efectuar predicciones sobre el comportamiento de
nuevos clientes.
Tu misión si decides aceptarla es que llegues a validar si es posible construir un
modelo que cumpla con este requerimiento, atravesando todas las fases clásicas de
un problema de machine learning (comprender el problema, analizar los datos,
entrenar y evaluar el modelo, qué se está teniendo en cuenta a la hora de realizar
predicciones, análisis de errores, etc.) utilizando buenas prácticas de programación
y explicando cada paso.

# Importação

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import statsmodels

In [12]:
df_bruto = pd.read_csv('../data/MeliDataset.csv', sep = ';')

In [13]:
df_bruto.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


# Descrição dos Dados

In [17]:
df_bruto.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

Dados do cliente # meli:
1 - idade (numérica)

2 - emprego: tipo de emprego (categoria: 'administrativo', 'operário', 'empreendedor', 'empregada doméstica',
'gestão', 'aposentado', 'autônomo', 'serviços', 'estudante', 'técnico', 'desempregado', 'desconhecido')

3 - civil: estado civil (categoria: 'divorciado', 'casado', 'solteiro', 'desconhecido'; observação:
'divorciado' significa divorciado ou viúvo)

4 - escolaridade (categoria: 'básico.4 anos', 'básico.6 anos', 'básico.9 anos', 'ensino médio', 'analfabeto',
'curso profissionalizante', 'diploma universitário', 'desconhecido')

5 - inadimplência: tem crédito em inadimplência? (categórico: 'não', 'sim', 'desconhecido')

6 - habitação: possui financiamento imobiliário? (categórico: 'não', 'sim', 'desconhecido')

7 - empréstimo: possui empréstimo pessoal? (categórico: 'não', 'sim', 'desconhecido')
Dados do último contato da campanha # meli:

8 - contato: tipo de comunicação do contato (categórico: 'celular', 'telefone')

9 - mês: mês do último contato do ano (categórico: 'jan', 'fev', 'mar', ..., 'nov', 'dez')

10 - dia_da_semana: dia da semana do último contato (categórico:
'seg', 'ter', 'qua', 'qui', 'sex')

11 - duração: duração do último contato, em segundos (numérico). Observação importante: este
atributo afeta bastante a meta de saída (por exemplo, se duração = 0, então y = 'não'). No entanto, a duração não é conhecida antes da chamada ser realizada. Além disso, após o término da chamada, y é obviamente conhecido. Portanto, essa entrada deve ser incluída apenas para fins de benchmark e deve ser descartada se a intenção for ter um modelo preditivo realista. # outros atributos:

12 - campanha: número de contatos realizados durante esta campanha e para este
cliente (numérico, inclui o último contato)

13 - pdays: número de dias decorridos desde o último contato do cliente em uma
campanha anterior (numérico; 999 significa que o cliente não foi contatado anteriormente)

14 - anterior: número de contatos realizados antes desta campanha e para este cliente
(numérico)

15 - poutcome: resultado da campanha de marketing anterior (categórico:
'fracasso','inexistente','sucesso')
# atributos de contexto social e econômico
16 - emp.var.rate: taxa de variação do emprego - indicador trimestral (numérico)
17 - cons.price.idx: índice de preços ao consumidor - indicador mensal (numérico)
18 - cons.conf.idx: índice de confiança do consumidor - indicador mensal (numérico)
19 - euribor3m: taxa euribor trimestral - indicador diário (numérico)
20 - nr.employed: número de funcionários - trimestral Indicador (numérico)
Variável de saída (meta desejada):
21 - s - o cliente aderiu a um "Mercado Crédito Ya"? (binário: 'sim', 'não')

In [27]:
print("Número de Linhas: ", df_bruto.shape[0])
print("Número de Colunas: ", df_bruto.shape[1])


Número de Linhas:  41188
Número de Colunas:  21


In [33]:
df_bruto['y'].value_counts()/len(df_bruto)

y
no     0.887346
yes    0.112654
Name: count, dtype: float64

Situação: dados desbalanceados com base na classe alvo.

Ação: faremos testes na construção do modelo com dados desbalanceados observando a métrica auprc e testaremos também com oversampling da classe minoritária.

In [20]:
df_bruto.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

In [21]:
df_bruto.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [22]:
df_bruto.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [23]:
num_attributes = df_bruto.select_dtypes(include=['int64', 'float64'])
num_attributes.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0


In [24]:
categorical_attributes = df_bruto.select_dtypes(include=['object'])
categorical_attributes.head()

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,nonexistent,no
1,services,married,high.school,unknown,no,no,telephone,may,mon,nonexistent,no
2,services,married,high.school,no,yes,no,telephone,may,mon,nonexistent,no
3,admin.,married,basic.6y,no,no,no,telephone,may,mon,nonexistent,no
4,services,married,high.school,no,no,yes,telephone,may,mon,nonexistent,no


In [26]:
num_attributes.describe().T

,count,mean,std,min,25%,50%,75%,max
age,41188.0,40.024060,10.421250,17.000,32.000,38.000,47.000,98.000
duration,41188.0,258.285010,259.279249,0.000,102.000,180.000,319.000,4918.000
campaign,41188.0,2.567593,2.770014,1.000,1.000,2.000,3.000,56.000
pdays,41188.0,962.475454,186.910907,0.000,999.000,999.000,999.000,999.000
previous,41188.0,0.172963,0.494901,0.000,0.000,0.000,0.000,7.000
emp.var.rate,41188.0,0.081886,1.570960,-3.400,-1.800,1.100,1.400,1.400
cons.price.idx,41188.0,93.575664,0.578840,92.201,93.075,93.749,93.994,94.767
cons.conf.idx,41188.0,-40.502600,4.628198,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,41188.0,3.621291,1.734447,0.634,1.344,4.857,4.961,5.045
nr.employed,41188.0,5167.035911,72.251528,4963.600,5099.100,5191.000,5228.100,5228.100


Situação: A variável "durantion" só pode ser adquirida após a ligação ser feita, logo ela não pode ser usada na construção de um modelo para prever se o cliente vai contratar o crédito antes da ligação ser feita.

Ação: Vamos remover a variável "duration".

In [29]:
categorical_attributes.apply( lambda x: x.unique().shape[0])

job            12
marital         4
education       8
default         3
housing         3
loan            3
contact         2
month          10
day_of_week     5
poutcome        3
y               2
dtype: int64

Situação: Variável alvo desbalanceada.

Ação:  Antes de seguirmos, vamos fazer uma divisão estratificada(10%) dos dados para fazer a validação e controle de overffiting do modelo e evitar vazamento de dados.

In [34]:
from sklearn.model_selection import train_test_split

X = df_bruto.drop(columns=['y'])
y = df_bruto['y']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.1,
    stratify=y,  # Garante proporções iguais entre as classes
    random_state=42  # Para reprodutibilidade
)

train_df = X_train.copy()
train_df['y'] = y_train

test_df = X_test.copy()
test_df['y'] = y_test

print("Train class distribution:\n", y_train.value_counts(normalize=True))
print("Test class distribution:\n", y_test.value_counts(normalize=True))

Train class distribution:
 y
no     0.887345
yes    0.112655
Name: proportion, dtype: float64
Test class distribution:
 y
no     0.887351
yes    0.112649
Name: proportion, dtype: float64


# Filtragem de Variáveis

# Análise Exploratório dos Dados

# Engenharia de Atributos

# Preparação dos Dados

# Seleção de Atributos

# Modelagem

# Ajuste de Hyper parâmetros